# 🏛️ Proyecto Integrador Nivel 2: Cortex Entity System

Hemos reconstruido Cortex desde cero utilizando **Programación Orientada a Objetos**.
En lugar de un script monolítico, ahora tenemos una arquitectura de componentes:

1.  **Clase `MemoryManager`:** Se encarga exclusivamente de leer y escribir en disco (JSON).
2.  **Clase `AnalysisTool`:** Encapsula la lógica de procesamiento de texto.
3.  **Clase `CortexKernel`:** El cerebro que orquesta todo. Hereda de una clase base `Agent`.

Esta estructura permite que, en el futuro, agregar una nueva capacidad (como búsqueda en internet) sea tan fácil como crear una nueva clase y registrarla, sin romper el código existente.

In [1]:
import json
import os
import datetime
from typing import List, Dict, Any

# --- COMPONENTE 1: GESTOR DE MEMORIA ---
class MemoryManager:
    """Se encarga exclusivamente de la persistencia de datos."""

    def __init__(self, filename: str = "memory.json"):
        self.filename = filename
        self._ensure_file_exists()

    def _ensure_file_exists(self):
        """Método 'privado' para garantizar que el archivo existe."""
        if not os.path.exists(self.filename):
            with open(self.filename, 'w', encoding='utf-8') as f:
                json.dump([], f) # Inicializar lista vacía

    def save(self, data: Dict[str, Any]):
        """Guarda un diccionario en el historial JSON."""
        history = self.load()
        history.append(data)
        with open(self.filename, 'w', encoding='utf-8') as f:
            json.dump(history, f, indent=4, ensure_ascii=False)

    def load(self) -> List[Dict[str, Any]]:
        """Carga todo el historial."""
        with open(self.filename, 'r', encoding='utf-8') as f:
            return json.load(f)

# --- COMPONENTE 2: HERRAMIENTAS (POLIMORFISMO) ---
class Tool:
    """Clase padre para todas las habilidades de Cortex."""
    def __init__(self, name):
        self.name = name

    def run(self, input_data: str) -> str:
        raise NotImplementedError("Las subclases deben implementar este método")

class TextAnalyzer(Tool):
    """Herramienta especializada en estadística de texto."""
    def run(self, text: str) -> dict:
        return {
            "tool": self.name,
            "words": len(text.split()),
            "uppercase": sum(1 for c in text if c.isupper()),
            "is_question": "?" in text
        }

# --- COMPONENTE 3: EL NÚCLEO (AGENTE) ---
class CortexKernel:
    """La entidad principal del Agente."""

    def __init__(self, name: str):
        self.name = name
        self.memory = MemoryManager() # Composición: El agente "tiene" memoria
        self.tools = {} # Diccionario de herramientas

        # Registrar herramientas nativas
        self.register_tool(TextAnalyzer("analyzer_v1"))

    def register_tool(self, tool: Tool):
        """Permite agregar nuevas habilidades dinámicamente."""
        self.tools[tool.name] = tool

    def _think(self, user_input: str) -> str:
        """Lógica interna de decisión."""
        # Lógica simple de enrutamiento
        if user_input.startswith("/analizar"):
            # Extraer texto: "/analizar Hola mundo" -> "Hola mundo"
            content = user_input.replace("/analizar", "").strip()
            if "analyzer_v1" in self.tools:
                result = self.tools["analyzer_v1"].run(content)
                return f"📊 REPORTE: {result}"

        elif user_input == "/memoria":
            history = self.memory.load()
            return f"🧠 Tengo {len(history)} recuerdos almacenados."

        return "Comando no entendido. Prueba '/analizar [texto]' o '/memoria'."

    def run_loop(self):
        """El bucle principal REPL."""
        print(f"🤖 {self.name} SYSTEM ONLINE (Arquitectura OOP)")
        print("Escribe '/salir' para terminar.")

        while True:
            try:
                user_input = input("\n👤 TU: ").strip()

                if user_input.lower() == "/salir":
                    print("🔴 Sistema Apagado.")
                    break

                # 1. Procesar
                response = self._think(user_input)

                # 2. Actuar (Imprimir)
                print(f"🤖 {self.name}: {response}")

                # 3. Memorizar (Estructurado)
                memory_entry = {
                    "timestamp": str(datetime.datetime.now()),
                    "input": user_input,
                    "response": response
                }
                self.memory.save(memory_entry)

            except Exception as e:
                print(f"❌ ERROR EN NÚCLEO: {e}")

# --- PUNTO DE ENTRADA ---
if __name__ == "__main__":
    # Instanciamos el Agente y corremos su bucle
    agent = CortexKernel("CORTEX-V2")
    agent.run_loop()

🤖 CORTEX-V2 SYSTEM ONLINE (Arquitectura OOP)
Escribe '/salir' para terminar.

👤 TU: /analizar hola inteligencia artificial como estas?
🤖 CORTEX-V2: 📊 REPORTE: {'tool': 'analyzer_v1', 'words': 5, 'uppercase': 0, 'is_question': True}

👤 TU: /memoria
🤖 CORTEX-V2: 🧠 Tengo 1 recuerdos almacenados.

👤 TU: analizar HOLA IA como estas?
🤖 CORTEX-V2: Comando no entendido. Prueba '/analizar [texto]' o '/memoria'.

👤 TU: /analizar HOLA IA como estas?
🤖 CORTEX-V2: 📊 REPORTE: {'tool': 'analyzer_v1', 'words': 4, 'uppercase': 6, 'is_question': True}

👤 TU: /memoria
🤖 CORTEX-V2: 🧠 Tengo 4 recuerdos almacenados.

👤 TU: salir
🤖 CORTEX-V2: Comando no entendido. Prueba '/analizar [texto]' o '/memoria'.

👤 TU: /salir
🔴 Sistema Apagado.
